In [52]:
from example.discretizer.utils import IsTrafficLightNearby, IsStopSignNearby, IsZebraNearby, LanePosition, FrontLeftObjects, FrontRightObjects
from pgeon.intention_introspector import IntentionIntrospector
from pgeon.desire import Desire
from pgeon.discretizer import Predicate
from example.discretizer.discretizer_d0 import AVDiscretizer
from example.discretizer.discretizer_d1 import AVDiscretizerD1
import pgeon.policy_graph as PG
from example.environment import SelfDrivingEnvironment

In [11]:

environment = SelfDrivingEnvironment(city='all')
discretizer = AVDiscretizer(environment, obj_discretization='binary')
nodes_path = f'example/dataset/data/policy_graphs/PG_trainval_Call_D0a_Wall_Tall_nodes.csv'
edges_path = f'example/dataset/data/policy_graphs/PG_trainval_Call_D0a_Wall_Tall_edges.csv'
pg = PG.PolicyGraph.from_nodes_and_edges(nodes_path, edges_path, environment, discretizer)

ii = IntentionIntrospector({})

    # %%
#iiap = ii.find_intentions(pg, 0.5)
#print(iiap.intention)


Number of nodes:             449
Number of edges:             2931


In [12]:
#NOTE: traffic light color can be also green or yellow
stop_traffic_light = Desire("stop_traffic_light", 1, {Predicate(IsTrafficLightNearby, [IsTrafficLightNearby.YES])})
ii.question4(pg, stop_traffic_light)
ii.question5(pg, stop_traffic_light,1)


How likely are you to find yourself in a state where you can fulfill your desire stop_traffic_light by performing the action 1?
Probability: 0.22083265659682766
How likely are you to perform your desirable action 1 when you are in the state region {IsTrafficLightNearby(YES)}?
Probability: 0.4989341484853398


In [14]:
stop_stop_sign = Desire("stop_stop_sign", 1, {Predicate(IsStopSignNearby, [IsStopSignNearby.YES])})
ii.question4(pg, stop_stop_sign)
ii.question5(pg, stop_stop_sign,1)

How likely are you to find yourself in a state where you can fulfill your desire stop_stop_sign by performing the action 1?
Probability: 0.0734123761572195
How likely are you to perform your desirable action 1 when you are in the state region {IsStopSignNearby(YES)}?
Probability: 0.15221294371929214


In [15]:
stop_zebra = Desire("stop_zebra", 1, {Predicate(IsZebraNearby, [IsZebraNearby.YES])})

ii.question4(pg, stop_zebra)
ii.question5(pg, stop_zebra,1)

How likely are you to find yourself in a state where you can fulfill your desire stop_zebra by performing the action 1?
Probability: 0.34199556060852143
How likely are you to perform your desirable action 1 when you are in the state region {IsZebraNearby(YES)}?
Probability: 0.3537164369487304


In [35]:
obstacle_avoidance_l = Desire("obstacle_avoidance_left", 7, {Predicate(FrontLeftObjects, [FrontLeftObjects(1, discretization="binary")]) })
obstacle_avoidance_r = Desire("obstacle_avoidance_right", 8, {Predicate(FrontRightObjects, [FrontRightObjects(1,discretization="binary")])})

ii.question4(pg, obstacle_avoidance_l)
ii.question5(pg, obstacle_avoidance_l, 7)
ii.question4(pg, obstacle_avoidance_r)
ii.question5(pg, obstacle_avoidance_r, 8)

How likely are you to find yourself in a state where you can fulfill your desire obstacle_avoidance_left by performing the action 7?
Probability: 0.77326620107195
How likely are you to perform your desirable action 7 when you are in the state region {FrontLeftObjects(YES)}?
Probability: 0.042201305346308664
How likely are you to find yourself in a state where you can fulfill your desire obstacle_avoidance_right by performing the action 8?
Probability: 0.814411780629094
How likely are you to perform your desirable action 8 when you are in the state region {FrontRightObjects(YES)}?
Probability: 0.037883096824323156


In [51]:
lane_change = Desire("lane_change", 8, {Predicate(LanePosition, [LanePosition.CENTER]) })

ii.question4(pg, lane_change)
ii.question5(pg, lane_change, 8)


How likely are you to find yourself in a state where you can fulfill your desire lane_change by performing the action 8?
Probability: 0.06036489632396723
How likely are you to perform your desirable action 8 when you are in the state region {LanePosition(CENTER)}?
Probability: 0.08429495935110046


In [54]:
environment = SelfDrivingEnvironment(city='all')
discretizer_configs = {
    'a': {'obj_discretizer': 'binary', 'vel_discretizer': 'binary' },
    'b': {'obj_discretizer': 'multiple', 'vel_discretizer': 'binary'},
    'c': {'obj_discretizer': 'multiple', 'vel_discretizer': 'multiple'}
    }

default_config = {'obj_discretizer': 'multiple', 'vel_discretizer': 'multiple'}

for discretizer_id in ['0a','0b','0c','1a','1b','1c']:
    config = default_config
    for key in discretizer_configs:
        if key in discretizer_id:
            config = discretizer_configs[key]
            break

    DiscretizerClass = AVDiscretizer if '0' in discretizer_id else AVDiscretizerD1
    discretizer = DiscretizerClass(
            environment,
            vel_discretization=config['vel_discretizer'],
            obj_discretization=config['obj_discretizer'],
            id=discretizer_id
        ) 
    nodes_path = f'example/dataset/data/policy_graphs/PG_trainval_Call_D{discretizer_id}_Wall_Tall_nodes.csv'
    edges_path = f'example/dataset/data/policy_graphs/PG_trainval_Call_D{discretizer_id}_Wall_Tall_edges.csv'
    print(f'DISCRETIZER {discretizer_id} ')
    pg = PG.PolicyGraph.from_nodes_and_edges(nodes_path, edges_path, environment, discretizer)

    stop_traffic_light = Desire("stop_traffic_light", 1, {Predicate(IsTrafficLightNearby, [IsTrafficLightNearby.YES])})
    ii.question4(pg, stop_traffic_light)
    ii.question5(pg, stop_traffic_light,1)
    stop_stop_sign = Desire("stop_stop_sign", 1, {Predicate(IsStopSignNearby, [IsStopSignNearby.YES])})
    ii.question4(pg, stop_stop_sign)
    ii.question5(pg, stop_stop_sign,1)
    stop_zebra = Desire("stop_zebra", 1, {Predicate(IsZebraNearby, [IsZebraNearby.YES])})
    ii.question4(pg, stop_zebra)
    ii.question5(pg, stop_zebra,1)
    lane_change = Desire("lane_change", 8, {Predicate(LanePosition, [LanePosition.CENTER]) })
    ii.question4(pg, lane_change)
    ii.question5(pg, lane_change, 8)
    lane_change = Desire("lane_change", 2, {Predicate(LanePosition, [LanePosition.CENTER]) })
    ii.question4(pg, lane_change)
    ii.question5(pg, lane_change, 2)


    
    print()

DISCRETIZER 0a 
How likely are you to find yourself in a state where you can fulfill your desire stop_traffic_light by performing the action 1?
Probability: 0.22083265659682766
How likely are you to perform your desirable action 1 when you are in the state region {IsTrafficLightNearby(YES)}?
Probability: 0.4989341484853398
How likely are you to find yourself in a state where you can fulfill your desire stop_stop_sign by performing the action 1?
Probability: 0.0734123761572195
How likely are you to perform your desirable action 1 when you are in the state region {IsStopSignNearby(YES)}?
Probability: 0.15221294371929214
How likely are you to find yourself in a state where you can fulfill your desire stop_zebra by performing the action 1?
Probability: 0.34199556060852143
How likely are you to perform your desirable action 1 when you are in the state region {IsZebraNearby(YES)}?
Probability: 0.3537164369487304
How likely are you to find yourself in a state where you can fulfill your desire